In [ ]:
from itertools import permutations
from math import floor, ceil
from copy import deepcopy
from functools import reduce

In [ ]:
def set_elt(tree, idx, val):
    if not hasattr(tree[idx[0]], "__iter__"):
        tree[idx[0]] = val
        return tree
    
    if len(idx) == 1:
        tree[idx[0]] = val
        return tree[idx[0]]
    
    return set_elt(tree[idx[0]], idx[1:], val)

In [ ]:
def get_elt(tree, idx):
    _idx = idx.copy()
    _idx.reverse()
    elt = tree
    while len(_idx) > 0 and hasattr(elt, "__iter__"):
        i = _idx.pop()
        elt = elt[i]
    return elt

In [ ]:
def add_in_direction(tree, idx, direction):
    _new_idx = int("".join(map(str, idx)), 2) + direction
    
    if _new_idx < 0:
        return set_elt(tree, idx + [0], 0)
    
    new_idx = list(map(int, list("{:04b}".format(_new_idx))))
    if len(new_idx) > 4:
        return tree
    
    while hasattr(get_elt(tree, new_idx), "__iter__"):
        new_idx += [int(direction == -1)]
                
    set_elt(tree, new_idx, get_elt(tree, new_idx) + get_elt(tree, idx + [int(direction == 1)]))
    return tree

In [ ]:
def explode(tree):
    for i in range(16):
        idx = list(map(int, list("{:04b}".format(i))))
        if hasattr(get_elt(tree, idx), "__iter__"):
            break

    if not hasattr(get_elt(tree, idx), "__iter__"):
        return tree
    
    add_in_direction(tree, idx, -1)
    add_in_direction(tree, idx, +1)
    set_elt(tree, idx, 0)
    
    return tree

In [ ]:
def split(tree):
    for i in range(16):
        idx = list(map(int, list("{:04b}".format(i))))
        elt = get_elt(tree, idx)
        if not hasattr(elt, "__iter__") and elt > 9:
            break
    
    elt = get_elt(tree, idx)
    if hasattr(elt, "__iter__") or elt <= 9:
        return tree

    set_elt(tree, idx, [floor(elt / 2), ceil(elt / 2)])
    
    return tree

In [ ]:
def add(tree_a, tree_b):
    tree = [deepcopy(tree_a), deepcopy(tree_b)]
    old_tree = []
    while old_tree != tree:        
        for op in [explode, split]:
            old_tree = deepcopy(tree)
            op(tree)
            if old_tree != tree:
                break
    return tree

In [ ]:
def magnitude(tree):
    if isinstance(tree, int):
        return tree
    
    return 3 * magnitude(tree[0]) + 2 * magnitude(tree[1])

In [ ]:
with open("input", "r") as f:
    snailfish_numbers = [eval(number.strip()) for number in f.readlines()]

### Part 1

In [ ]:
magnitude(reduce(add, snailfish_numbers))

### Part 2

In [ ]:
max(magnitude(add(*pair)) for pair in permutations(snailfish_numbers, 2))